In [ ]:
class ArbreBinaire:
    def __init__(self,cle =None , gauche = None, droite = None,parent = None):
        self.cle = cle
        self.gauche = gauche
        self.droite = droite
        self.parent = parent

In [ ]:
class TrieBin :
    def __init__(self, tab , c = None):
        self.tab = tab  # Exemple : tab = [('a','01'),('b','11'),('c','00'),('d','10')]
        if c is None :
            self.racine = None
        else :  #On suppose que c est un caractère et que tab est un tableau de couples (caractère, caractère binaire)
            self.racine = ArbreBinaire(c)
            
    def ArbreVide(self):
        return ArbreBinaire()

    def car(self, c , i):
        for k , v in self.tab.items():
            if k == c and len(v)-1 >= i:
                res= v[i]
                return int(res)
        return None

    def TrieBin_Ajout(self, c):
        self.racine = self.TrieBin_Ajout_rang(self.racine, c, 0)
        return self
    

    def TrieBin_Ajout_rang(self,arbre, c, i):
        if arbre is None:
            return ArbreBinaire(c)
        new_node = ArbreBinaire(c,None,None,arbre)
        if arbre.gauche is None and arbre.droite is None:
            if arbre.cle == c:
                return arbre
            else:
                arbre = self.split(arbre, c, i)
                return arbre
        if self.car(c, i) == 0:
            if arbre.gauche is None:
                arbre.gauche = new_node
                new_node.parent = arbre
            else:
                arbre.gauche = self.TrieBin_Ajout_rang(arbre.gauche , c, i + 1 )
        else:
            if arbre.droite is None:
                arbre.droite = new_node
                new_node.parent = arbre
            else:
                arbre.droite = self.TrieBin_Ajout_rang(arbre.droite, c, i + 1 )
        return arbre


    def split(self, arbre , c2, i):
        if self.car(arbre.cle,i) == self.car(c2,i) == 0:
            return ArbreBinaire( '*' , self.split(arbre, c2, i+1), None, arbre)
        if self.car(arbre.cle,i) == self.car(c2,i) == 1:
            return ArbreBinaire( '*' , None, self.split(arbre, c2, i+1), arbre)
        if self.car(arbre.cle,i) == 0 and self.car(c2,i) == 1:
            return ArbreBinaire( '*' , ArbreBinaire(arbre.cle), ArbreBinaire(c2), arbre)
        if self.car(arbre.cle,i) == 1 and self.car(c2,i) == 0:
            return ArbreBinaire( '*' , ArbreBinaire(arbre.cle), ArbreBinaire(arbre.cle), arbre)
        


    def TrieBin_Recherche(self, c):
        return self.TrieBin_Recherche_rang(self.racine, c, 0)
    
    def TrieBin_Recherche_rang(self, arbre, c, i):
        if arbre is None:
            return False
        if arbre.cle == c:
            return True
        if self.car(c, i) == 0:
            return self.TrieBin_Recherche_rang(arbre.gauche, c, i + 1)
        else:
            return self.TrieBin_Recherche_rang(arbre.droite, c, i + 1)


    def TrieBin_Suppression(self, c):
        return self.TrieBin_Suppression_rang(self.racine, c, 0)
    
    def TrieBin_Suppression_rang(self, arbre, c, i):
        if arbre is None:
            return None
        if arbre.cle == c:
            if arbre.gauche is None and arbre.droite is None:
                return None
            if arbre.gauche is None:
                return arbre.droite
            if arbre.droite is None:
                return arbre.gauche
            else:
                tmp = self.TrieBin_min(arbre.droite)
                #tmpMAJ = arbre
                arbre.cle = tmp.cle
                arbre.droite = self.TrieBin_Suppression_rang(arbre.droite, tmp.cle, i + 1)
                
                self.TrieBin_MiseAJour(arbre)
                return arbre
        if self.car(c, i) == 0:
            arbre.gauche = self.TrieBin_Suppression_rang(arbre.gauche, c, i + 1)
            self.TrieBin_MiseAJour(arbre)
        else:
            arbre.droite = self.TrieBin_Suppression_rang(arbre.droite, c, i + 1)
            self.TrieBin_MiseAJour(arbre)
        return arbre
    
    def TrieBin_min(self, arbre):
        cur = arbre
        while cur.gauche is not None:
            cur = cur.gauche
        return cur

    def TrieBin_MiseAJour(self, arbre):
        if arbre.parent is None:
            return
        if arbre.parent.cle == '*':
            if arbre.parent.gauche == arbre and arbre.parent.droite is None:
                arbre.parent.cle = arbre.gauche.cle if arbre.gauche is not None else arbre.droite.cle
                arbre.parent.gauche = arbre.gauche if arbre.gauche is not None else arbre.droite
            elif arbre.parent.droite == arbre and arbre.parent.gauche is None:
                arbre.parent.cle = arbre.gauche.cle if arbre.gauche is not None else arbre.droite.cle
                arbre.parent.droite = arbre.gauche if arbre.gauche is not None else arbre.droite



    # def TrieBin_MiseAJour(self, arbre):
    #     if arbre.parent.cle =='*' : 
    #         if arbre.parent.gauche == arbre and arbre.parent.droite is None:
    #             arbre.parent.cle = arbre.parent.gauche.cle
    #             arbre.parent.gauche = None
    #         if arbre.parent.gauche is None and arbre.parent.droite == arbre:
    #             arbre.parent.cle = arbre.parent.droite.cle
    #             arbre.parent.droite = None
            


    # def TrieBin_MiseAJour(self, c):
    #     trouver = self.TrieBin_Find(self.racine, c, 0)
    #     if trouver is not None:
    #         print("trouverrrrrrrrrrrrr",trouver.cle)
    #         if trouver.parent.cle =='*' : 
    #             if trouver.parent.gauche is not None and trouver.parent.droite is None:
    #                 trouver.parent.cle = trouver.parent.gauche.cle
    #                 trouver.parent.gauche = None
    #             if trouver.parent.gauche is None and trouver.parent.droite is not None:
    #                 trouver.parent.cle = trouver.parent.droite.cle
    #                 trouver.parent.droite = None

    def TrieBin_Largeur(self, arbre):
        if arbre is None:
            return None
        if arbre.gauche is None and arbre.droite is None and arbre.cle == '*':
            return None
        
        else:
            self.TrieBin_Largeur(arbre.gauche)
            self.TrieBin_Largeur(arbre.droite)
            
        





    def TrieBin_MiseAJour2(self, c):
        trouver = self.TrieBin_Find(self.racine, c, 0)
        if trouver is not None:
            if trouver.parent is None : 
                if trouver.gauche is not None and trouver.droite is None:
                    trouver.parent.cle = trouver.gauche.cle
                    trouver.parent.gauche = None
                if trouver.gauche is None and trouver.droite is not None:
                    trouver.parent.cle = trouver.droite.cle
                    trouver.parent.droite = None
            elif trouver.parent.gauche == trouver:
                trouver.parent.gauche = trouver.gauche if trouver.gauche is not None else trouver.droite
            else:
                trouver.parent.droite = trouver.gauche if trouver.gauche is not None else trouver.droite



    def TrieBin_Find(self, arbre, c , i):
        if arbre is None:
            return None
        if arbre.cle == c:
            return arbre
        if self.car(c, i) == 0:
            return self.TrieBin_Find(arbre.gauche, c, i + 1)
        else:
            return self.TrieBin_Find(arbre.droite, c, i + 1)
        

    def TrieBin_Construction(self):
        for i in self.tab.keys():
            self.TrieBin_Ajout(i)
        return self


    def afficher_arbre(self , arbre, niveau=0):
        if arbre is not None:
            self.afficher_arbre(arbre.droite, niveau + 1)
            if niveau > 0:
                print('   ' * (niveau - 1) + '|--', end='')
            print(str(arbre.cle))
            self.afficher_arbre(arbre.gauche, niveau + 1)

In [ ]:
def test_TrieBin():
    tab = {'A': '00001', 'S': '10011', 'E': '00101', 'R': '10010', 'C': '00011', 'H': '01000', 'I': '01001', 'N': '01110', 'G': '00111', 'X': '11000', 'M': '01101', 'P': '10000', 'L': '01100'}
    triebin = TrieBin(tab)
    triebin = triebin.TrieBin_Construction()
    triebin.afficher_arbre(triebin.racine)

    print("----------------------")
    for k in tab.keys():
        trouver = triebin.TrieBin_Recherche(k)
        print("trouver",k,"?",trouver)

    trouver = triebin.TrieBin_Recherche('89')
    print("trouver 89 ?",trouver)

    print("------------Suppriner 'E'-----------")
    suppr = triebin.TrieBin_Suppression('E')
    trouver = triebin.TrieBin_Recherche('E')
    print("trouver E ?",not(trouver))
    triebin.TrieBin_Largeur(triebin.racine)
    triebin.afficher_arbre(triebin.racine)

    print("------------Supprimer 'A'-----------")
    suppr = triebin.TrieBin_Suppression('A')
    trouver = triebin.TrieBin_Recherche('A')
    print("trouver A ?",not(trouver))
    triebin.afficher_arbre(triebin.racine)

    print("------------Supprimer 'C'-----------")
    suppr = triebin.TrieBin_Suppression('C')
    trouver = triebin.TrieBin_Recherche('C')
    print("trouver C ?",not(trouver))
    triebin.TrieBin_Largeur(triebin.racine)
    triebin.afficher_arbre(triebin.racine)




test_TrieBin()